In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from operator import itemgetter
from datetime import date, datetime
import time

# preprocessing

In [3]:
def preprocessing(stock, simul):

    stock['simul'] = simul
    
    per_list = [0,]
    for i in range(1,len(stock)):
        plus_per = ((stock.simul[i]/stock.simul[i-1])-1) * 100
        per_list.append(plus_per)
    stock["simul_per"] = per_list
    
    datetime_list = [datetime.strptime(i,"%Y-%m-%d").date() for i in stock.date]
    stock["datetime"] = datetime_list
    
    return stock


# metrics

## beta

In [4]:
def beta_func(stock):

    # portfolio cov
    port_cov = stock.close.cov(stock.simul)

    # 시장 수익률 분산
    market_cov = np.var(stock.close)

    # beta market
    beta_market = market_cov / market_cov
    
    # beta port
    beta_port = port_cov / market_cov
    
    return beta_market, beta_port


## sharp

In [5]:
def sharp_func(stock):
        
    per_list = [0,]
    for i in range(1,len(stock)):
        plus_per = ((stock.simul[i]/stock.simul[i-1])-1) * 100
        per_list.append(plus_per)
    stock["simul_per"] = per_list
    
    market_diff = (stock.close[len(stock)-1] - stock.close[1]) / stock.close[1]
    simul_diff = (stock.simul[len(stock)-1] - stock.simul[1]) / stock.simul[1]
    
    sharp_market = market_diff / np.std(stock.per)
    sharp_simul = simul_diff / np.std(stock.simul_per)
    
    return sharp_market, sharp_simul


## sortino

In [6]:
def sortino_func(stock):
    sortino_close = [i if i < 0 else 0 for i in stock.per]
    sortino_simul = [i if i < 0 else 0 for i in stock.simul_per]
    
    market_diff = (stock.close[len(stock)-1] - stock.close[1]) / stock.close[1]
    simul_diff = (stock.simul[len(stock)-1] - stock.simul[1]) / stock.simul[1]
    
    sortino_market = market_diff / np.std(np.abs(sortino_close))    
    sortino_minus = simul_diff / np.std(np.abs(sortino_simul))
    
    return sortino_market, sortino_minus


## 최대낙폭 (MDD)

In [7]:
def MDD_func(stock):
    
    MDD_list = []
    for i in range(0,len(stock.per)-1):
        onepoint = stock.close[i]
        drawdown = np.min(stock.close[(i+1):])
        MDD = 100 - (drawdown/onepoint)*100
        MDD_list.append(MDD)
    MDD_market = MDD_list[np.argmax(MDD_list)]

    MDD_list = []
    for i in range(0,len(stock.simul)-1):
        onepoint = stock.simul[i]
        drawdown = np.min(stock.simul[(i+1):])
        MDD = 100 - (drawdown/onepoint)*100
        MDD_list.append(MDD)
    MDD_simul = MDD_list[np.argmax(MDD_list)]
    
    return MDD_market, MDD_simul


In [8]:
def plot_measure(stock):
    
    beta_values = beta_func(stock)
    sharp_values = sharp_func(stock)
    sortino_values = sortino_func(stock)
    MDD_values = MDD_func(stock)
    
    x = range(2)
    port = ["market","portfolio"]
    value_list = [beta_values, sharp_values, sortino_values, MDD_values]
    value_name = ["beta","sharp","sortino","MDD"]
    
    fig, axs = plt.subplots(1,4,figsize = [11,3.5])
    for i,value,name in zip(range(4), value_list, value_name):
        axs[i].bar(x,value,color=['tab:orange','tab:blue'],width = 0.5); 
        axs[i].set_xticks(x); 
        axs[i].set_xticklabels(port);
        axs[i].set_title(name,fontweight="bold")
    axs[0].axhline(y=1,color='red',linestyle='--')
    plt.tight_layout()
    plt.show()
    
    print("beta    :" + str(beta_values))
    print("sharp   :" + str(sharp_values))
    print("sortino :" + str(sortino_values))
    print("MDD     :" + str(MDD_values))
    

In [9]:
def plot_profit(stock, anomaly_list):
    fig, ax = plt.subplots(figsize = [13, 3])
    ax.plot(stock.datetime,stock.simul,label='portfolio')
    ax.plot(stock.datetime,stock.close,label='market')
    for i in anomaly_list:
        ax.axvspan(i[0], i[1], alpha=0.3, color='darkgray')
    ax.set_title('price',fontweight="bold")
    plt.legend(loc="upper left")
    
    plt.show()
    

In [10]:
def plot_alpha(stock, anomaly_list):
    fig, ax = plt.subplots(figsize = [13,3])
    # 포트폴리오 수익률
    ax.plot(stock.datetime,list(np.array(stock.simul)-np.array(stock.close)),label = 'portfolio')
    # 시장수익률
    ax.axhline(y=0,color='tab:orange',linestyle='--',label = 'market')
    for i in anomaly_list:
        ax.axvspan(i[0], i[1], alpha=0.3, color='darkgray')

    # 구간
    ax.set_title('alpha',fontweight="bold")
    ax.legend(loc="upper left")
    plt.show()
    

# plot all

In [ ]:
from minio import Minio

client = Minio(
    "192.168.0.207:8801",  # URI
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False)

In [11]:
def plot_all(stock, anomaly_list, index_name):
    
    fig = plt.figure(figsize = [15,10])

    # plot 1
    ax1 = fig.add_subplot(3,1,1)
    ax1.plot(stock.datetime,stock.simul,label='portfolio')
    ax1.plot(stock.datetime,stock.close,label='market')
    if len(anomaly_list) >  0:
        for i in anomaly_list:
            ax1.axvspan(i[0], i[1], alpha=0.3, color='darkgray')
    ax1.set_title('price',fontweight="bold")
    plt.legend(loc="upper left")

    
    # plot 2
    ax2 = fig.add_subplot(3,1,2)
    ax2.plot(stock.datetime,list(np.array(stock.simul)-np.array(stock.close)),label = 'portfolio')
    ax2.axhline(y=0,color='tab:orange',linestyle='--',label = 'market')
    if len(anomaly_list) >  0:
        for i in anomaly_list:
            ax2.axvspan(i[0], i[1], alpha=0.3, color='darkgray')
    ax2.set_title('alpha',fontweight="bold")

    
    # plot 3
    x = range(2)
    port = ["market","portfolio"]
    beta_values = beta_func(stock)
    sharp_values = sharp_func(stock)
    sortino_values = sortino_func(stock)
    MDD_values = MDD_func(stock)

    ax3 = fig.add_subplot(3,4,9)
    ax3.bar(x,beta_values,color=['tab:orange','tab:blue'],width = 0.5); 
    ax3.set_xticks(x); 
    ax3.set_xticklabels(port);
    ax3.set_title("beta",fontweight="bold")
    ax3.axhline(y=1,color='red',linestyle='--')

    ax4 = fig.add_subplot(3,4,10)
    ax4.bar(x,sharp_values,color=['tab:orange','tab:blue'],width = 0.5); 
    ax4.set_xticks(x); 
    ax4.set_xticklabels(port);
    ax4.set_title("sharp",fontweight="bold")

    ax5 = fig.add_subplot(3,4,11)
    ax5.bar(x,sortino_values,color=['tab:orange','tab:blue'],width = 0.5); 
    ax5.set_xticks(x); 
    ax5.set_xticklabels(port);
    ax5.set_title("sortino",fontweight="bold")

    ax6 = fig.add_subplot(3,4,12)
    ax6.bar(x,MDD_values,color=['tab:orange','tab:blue'],width = 0.5); 
    ax6.set_xticks(x); 
    ax6.set_xticklabels(port);
    ax6.set_title("MDD",fontweight="bold")

    fig.suptitle('{}'.format(index_name), fontweight ="bold") 
    
    plt.savefig('./DATA/portfolio_metric/{}.png'.format(index_name))
    result = client.fput_object("stock-dataset", "portfolio_metric/{}.png".format(index_name), "./DATA/portfolio_metric/{}.png".format(index_name))
    
    